# Load the repository and install librairies

---



We clone the repository of the main code,

In [1]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [ ]:
!git clone https://github.com/Isaaclabe/DGD-Dynamic-3D-Gaussians-Distillation.git --recursive
%cd /content/DGD-Dynamic-3D-Gaussians-Distillation

We install the libraries,

In [ ]:
!pip install -q plyfile
!pip install git+https://github.com/openai/CLIP.git
!pip install timm
!pip install -r /content/DGD-Dynamic-3D-Gaussians-Distillation/requirements.txt

# Setup the libraries for the differentiable Gaussian Rasterizer, simple Knn algorithm and the Lseg minimal model

---

In [6]:
import shutil
import os

Setup for the differentiable Gaussian Rasterizer, if you want to use the DINOv2 foundation model you need to set NUM_SEMANTIC_CHANNELS = 384 in the file: /content/DGD-Dynamic-3D-Gaussians-Distillation/submodules/diff-gaussian-rasterization/cuda_rasterizer/config.h. But if you want to use the Lseg-CLIP foundation model you need to put NUM_SEMANTIC_CHANNELS = 512.


In [ ]:
%cd /content/DGD-Dynamic-3D-Gaussians-Distillation/submodules/diff-gaussian-rasterization
!python setup.py build_ext

In [ ]:
os.makedirs("/content/DGD-Dynamic-3D-Gaussians-Distillation/diff_gaussian_rasterization", exist_ok=True)
source_path_1 = "/content/DGD-Dynamic-3D-Gaussians-Distillation/submodules/diff-gaussian-rasterization/build/lib.linux-x86_64-cpython-310/diff_gaussian_rasterization/_C.cpython-310-x86_64-linux-gnu.so"
source_path_2 = "/content/DGD-Dynamic-3D-Gaussians-Distillation/submodules/diff-gaussian-rasterization/diff_gaussian_rasterization/__init__.py"
destination_folder_1 = "/content/DGD-Dynamic-3D-Gaussians-Distillation/diff_gaussian_rasterization"
shutil.move(source_path_1, destination_folder_1)
shutil.move(source_path_2, destination_folder_1)

Setup for the simple Knn algorithm,


In [ ]:
%cd /content/DGD-Dynamic-3D-Gaussians-Distillation/submodules/simple-knn
!python setup_knn.py build_ext

In [ ]:
os.makedirs("/content/DGD-Dynamic-3D-Gaussians-Distillation/simple_knn", exist_ok=True)
source_path_3 = "/content/DGD-Dynamic-3D-Gaussians-Distillation/submodules/simple-knn/build/lib.linux-x86_64-cpython-310/simple_knn/_C.cpython-310-x86_64-linux-gnu.so"
destination_folder_2 = "/content/DGD-Dynamic-3D-Gaussians-Distillation/simple_knn"
shutil.move(source_path_3, destination_folder_2)

Setup for the Lseg minimal model,


In [ ]:
%cd /content/DGD-Dynamic-3D-Gaussians-Distillation/lseg_minimal
!python setup.py build develop
!rm -r /content/DGD-Dynamic-3D-Gaussians-Distillation/lseg_minimal/lseg
!mv /content/DGD-Dynamic-3D-Gaussians-Distillation/lseg_minimal/build/lib/lseg/ /content/DGD-Dynamic-3D-Gaussians-Distillation/lseg_minimal/
!rm -r /content/DGD-Dynamic-3D-Gaussians-Distillation/lseg_minimal/build

We donwload the Lseg-minimal model,

In [ ]:
!git clone https://huggingface.co/IsaacLabe/Lseg_minimal_model

# Load the datasets

---



In [ ]:
%cd /content/DGD-Dynamic-3D-Gaussians-Distillation

!git clone https://huggingface.co/IsaacLabe/data_hyperNerf
!unzip /content/DGD-Dynamic-3D-Gaussians-Distillation/data_hyperNerf/misc_split-cookie.zip

# Train

---



Use the DINOv2 foundation model,

In [ ]:
!python train.py -s "/content/DGD-Dynamic-3D-Gaussians-Distillation/split-cookie" -m "/content/DGD-Dynamic-3D-Gaussians-Distillation/" --fundation_model "DINOv2" --semantic_dimension 384

Use the Lseg-CLIP foundation model,

In [ ]:
!python train.py -s "/content/DGD-Dynamic-3D-Gaussians-Distillation/split-cookie" -m "/content/DGD-Dynamic-3D-Gaussians-Distillation/" --Lseg_model_path "/content/DGD-Dynamic-3D-Gaussians-Distillation/lseg_minimal/Lseg_minimal_model/lseg_minimal_e200.ckpt" --fundation_model "Lseg_CLIP" --semantic_dimension 512 --loss_reduce 10

# Render

---



Use the DINOv2 foundation model,

In [ ]:
!python render.py -s "/content/DGD-Dynamic-3D-Gaussians-Distillation/split-cookie" -m "/content/DGD-Dynamic-3D-Gaussians-Distillation/" --fundation_model "DINOv2" --semantic_dimension 384 --iterations 40_000 --frame 539 --novel_views -1 --points "(270,370)" "(200,450)" --thetas "0.55" "0.7"

Use the Lseg-CLIP foundation model,

In [ ]:
!python render.py -s "/content/DGD-Dynamic-3D-Gaussians-Distillation/split-cookie" -m "/content/DGD-Dynamic-3D-Gaussians-Distillation/" --Lseg_model_path "/content/DGD-Dynamic-3D-Gaussians-Distillation/lseg_minimal/Lseg_minimal_model/lseg_minimal_e200.ckpt" --fundation_model "Lseg_CLIP" --semantic_dimension 384 --iterations 40_000 --frame 539 --novel_views 100 --prompt "Cookie" --thetas "0.55"